In [12]:
import numpy as np
from PIL import Image
import random

In [13]:
def load_and_scale_images(image_path, watermark_path):
    image = Image.open(image_path).convert('L')  
    image = image.resize((512, 512))
    
    image_array = np.array(image)
    
    watermark = Image.open(watermark_path).convert('L')
    watermark = watermark.resize((32, 32))
    watermark_array = np.array(watermark) > 128  
    
    return image_array, watermark_array, image, watermark

In [14]:
img_array,wtr_array,img,wtr=load_and_scale_images('lena.png','watermark_image.webp')

In [15]:
wtr.save('watermark_32x32.png')

In [16]:
blocks = []
original_indices = []
complexity_scores = []

In [38]:
def svd_decomposition(image_array, block_size=8):
    for i in range(0, image_array.shape[0], block_size):
        for j in range(0, image_array.shape[1], block_size):
            block = image_array[i:i+block_size, j:j+block_size]

            U, D, Vt = np.linalg.svd(block)
            
            complexity_score = np.sum(D ** 2)
            blocks.append((U, D, Vt))
            original_indices.append((i, j))
            complexity_scores.append(complexity_score)
            #print(f"Block ({i}, {j}): {complexity_score}")
    
    threshold = np.percentile(complexity_scores,95)
 
    complex_blocks = [(blocks[idx], original_indices[idx]) for idx, score in enumerate(complexity_scores) if score >= threshold]
    
    return complex_blocks

In [39]:
cb=svd_decomposition(img_array)

In [40]:
len(cb)

1640

In [10]:
b0=[]
def embed_watermark_in_blocks(complex_blocks, watermark_array, seed=42,threshold=0.01):
    random.seed(seed)
    watermark_bits = watermark_array.flatten()
    
    for bit in watermark_bits:
        block_idx = random.randint(0, len(complex_blocks) - 1)
        b0.append(block_idx)
        (U, D, Vt), (i, j) = complex_blocks[block_idx]
        
        if bit == 1:
            if U[0, 0] <= U[1, 0]: 
                U[0, 0] += 5e-2
                U[1, 0] -= 5e-2
            
            if U[0, 0] - U[1, 0] < threshold:
                U[0, 0] += threshold / 2
                U[1, 0] -= threshold / 2
        
        else:
            if U[0, 0] >= U[1, 0]: 
                U[0, 0] -= 5e-2
                U[1, 0] += 5e-2
            
            if U[1, 0] - U[0, 0] < threshold:
                U[0, 0] -= threshold / 2
                U[1, 0] += threshold / 2
    
        complex_blocks[block_idx] = (U, D, Vt), (i, j)
    
    return complex_blocks

In [11]:
def reconstruct_image(blocks, original_indices, block_size=8):
    reconstructed_image = np.zeros((512, 512))
    
    for (U, D, Vt), (i, j) in zip(blocks, original_indices):
        reconstructed_block = np.dot(U, np.dot(np.diag(D), Vt))
        reconstructed_image[i:i+block_size, j:j+block_size] = reconstructed_block
    
    return reconstructed_image